# STORD Case Study Solution

## (1) Min-Cost Model

In [2]:
from gurobipy import * 
model = Model("STORD")

Academic license - for non-commercial use only


### Making Indexes

In [4]:
#index i 
customers = [0,1,2,3,4,5,6]
customer_demands = {0:36, 1:42, 2:34, 3:50, 4:27, 5:30, 6:43}

#index j 
warehouses = [0,1,2,3,4,5]
warehouse_fixed_costs = {0:321420, 1:350640, 2:379860, 3:401775, 4:350640, 5:336030}

#index ij 
distance = [ [18,21,27,16,31,18],
			[23,18,18,23,20,17],
			[19,17,17,9,18,29],
			[21,23,20,31,19,21],
			[24,11,23,21,10,22],
			[17,18,9,23,17,18],
			[9,20,18,10,18,8] ]

An assumption I make here is that demand will be constant for the next 4 years.

In [5]:
cost_per_milepallet = 0.06 * 365 * 4

### Make Decision Variables 

In [6]:
x = model.addVars(customers, warehouses,  vtype = GRB.BINARY, name = "x" )
y = model.addVars(warehouses,  vtype = GRB.BINARY, name = "y" )

### Make Constraints

In [7]:
#Each customer is served by one warehouse 
model.addConstrs( quicksum(x[i,j] for j in warehouses) == 1 for i in customers )

#If a ware house is not open you cannot use the routes associated with it 
model.addConstrs( x[i,j] <= y[j] for i in customers for j in warehouses)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,


### Make Objective

In [8]:
transportation_cost = cost_per_milepallet * quicksum( x[i,j] * customer_demands[i] * distance[i][j] for i in customers for j in warehouses )
cost_of_openWH  = quicksum(warehouse_fixed_costs[j] * y[j] for j in warehouses) 

total_cost = transportation_cost + cost_of_openWH

model.setObjective(total_cost, GRB.MINIMIZE)

### Printing Solutions 

In [9]:
#solve 
model.optimize()

if model.status == GRB.Status.OPTIMAL:
	print()
	print('\nOptimal solution found in {}s'.format(round(model.Runtime,2)))
	print('\nThe total cost is {}'.format(model.objVal))
	print('\nThe optimal solution is:')
	x_sol = model.getAttr('x', x) # get the optimal solution
	
	v = model.getVars()
	
	for i in range(len(v)):
		if v[i].X != 0:
			print(v[i].varName, v[i].X)


Optimize a model with 49 rows, 48 columns and 126 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1777468.8000
Presolve time: 0.00s
Presolved: 49 rows, 48 columns, 126 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)

Root relaxation: objective 7.467180e+05, 43 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    746718.00000 746718.000  0.00%     -    0s

Explored 0 nodes (43 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 746718 1.77747e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.467180000000e+05, best bound 7.467180000000e+05, gap 0.00

### Calculate Total Miles

In [11]:
total_miles = 0
for i in range(7):
	total_miles = total_miles + distance[i][0]

print("Total Miles is" ,total_miles)

Total Miles is 131


## (2) Two Warehouses Model 

### We will add a new constraint 

In [12]:
#Open at least 2 WH 
model.addConstr ( quicksum( y[j] for j in warehouses) == 2 )

<gurobi.Constr *Awaiting Model Update*>

### Solve 

In [13]:
#solve 
model.optimize()

if model.status == GRB.Status.OPTIMAL:
	print()
	print('\nOptimal solution found in {}s'.format(round(model.Runtime,2)))
	print('\nThe total cost is {}'.format(model.objVal))
	print('\nThe optimal solution is:')
	x_sol = model.getAttr('x', x) # get the optimal solution
	
	v = model.getVars()
	
	for i in range(len(v)):
		if v[i].X != 0:
			print(v[i].varName, v[i].X)
	#pprint(x_sol)
 
    
model.write('STORD.lp')

Optimize a model with 50 rows, 48 columns and 132 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]

MIP start did not produce a new incumbent solution
MIP start violates constraint R49 by 1.000000000

Found heuristic solution: objective 1188861.6000
Presolve time: 0.00s
Presolved: 50 rows, 48 columns, 132 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)

Root relaxation: objective 1.041275e+06, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1041274.8000 1041274.80  0.00%     -    0s

Explored 0 nodes (23 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 1.04127e+06 1.18886e+06 

Optimal sol

### Calculate Total Miles 

In [14]:
total_miles = distance[0][5] + distance[1][5] + distance[2][4] + distance[3][4] + distance[4][4] + distance[5][4] + distance[6][5]
print("Total Miles is" ,total_miles)

Total Miles is 107


## (3) Three Warehouses Model 

### Remove constraint from 2WH model and Add Constraints

In [19]:
model.remove(model.getConstrs())

#Each customer is served by one warehouse 
model.addConstrs( quicksum(x[i,j] for j in warehouses) == 1 for i in customers )

#If a ware house is not open you cannot use the routes associated with it 
model.addConstrs( x[i,j] <= y[j] for i in customers for j in warehouses)

#Open at least 3 WH 
model.addConstr ( quicksum( y[j] for j in warehouses) == 3 )

<gurobi.Constr *Awaiting Model Update*>

### Solve

In [20]:
#solve 
model.optimize()

if model.status == GRB.Status.OPTIMAL:
	print()
	print('\nOptimal solution found in {}s'.format(round(model.Runtime,2)))
	print('\nThe total cost is {}'.format(model.objVal))
	print('\nThe optimal solution is:')
	x_sol = model.getAttr('x', x) # get the optimal solution
	
	v = model.getVars()
	
	for i in range(len(v)):
		if v[i].X != 0:
			print(v[i].varName, v[i].X)
	#pprint(x_sol)
 
    
model.write('STORD.lp')

Optimize a model with 50 rows, 48 columns and 132 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start did not produce a new incumbent solution
MIP start violates constraint R100 by 1.000000000

Found heuristic solution: objective 1450207.2000
Presolve time: 0.00s
Presolved: 50 rows, 48 columns, 132 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)

Root relaxation: objective 1.362695e+06, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1362694.8000 1362694.80  0.00%     -    0s

Explored 0 nodes (14 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 1.36269e+06 1.45021e+06 

Optimal so

### Calculate Total Miles 

In [22]:
total_miles = 0
total_miles = distance[0][0] + distance[1][5] + distance[2][4] + distance[3][4] + distance[4][4] + distance[5][0] + distance[6][5]
print("Total Miles is" ,total_miles)

Total Miles is 107


## (4) Min-Distance Model 

### Remove constraint from 3WH model and Add Constraints

In [23]:
model.remove(model.getConstrs())

#Each customer is served by one warehouse 
model.addConstrs( quicksum(x[i,j] for j in warehouses) == 1 for i in customers )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

### Make new Objective 

In [24]:
transportation_cost =  quicksum( x[i,j] * distance[i][j] for i in customers for j in warehouses )

model.setObjective(transportation_cost, GRB.MINIMIZE)

### Solve

In [26]:
#solve 
model.optimize()

if model.status == GRB.Status.OPTIMAL:
	print()
	print('\nOptimal solution found in {}s'.format(round(model.Runtime,2)))
	print('\nThe total mile is {}'.format(model.objVal))
	print('\nThe optimal solution is:')
	x_sol = model.getAttr('x', x) # get the optimal solution
	
	v = model.getVars()
	
	for i in range(len(v)):
		if v[i].X != 0:
			print(v[i].varName, v[i].X)
	#pprint(x_sol)
 
    
model.write('STORD.lp')

Optimize a model with 7 rows, 48 columns and 42 nonzeros
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start with objective 1e+100

Presolve removed 7 rows and 48 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 88 107 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.800000000000e+01, best bound 8.800000000000e+01, gap 0.0000%


Optimal solution found in 0.02s

The total mile is 88.0

The optimal solution is:
x[0,3] 1.0
x[1,5] 1.0
x[2,3] 1.0
x[3,4] 1.0
x[4,4] 1.0
x[5,2] 1.0
x[6,5] 1.0


### Find Total Cost

In [28]:
total_miles = distance[0][3] + distance[1][5] + distance[2][3] + distance[3][4] + distance[4][4] + distance[5][2] + distance[6][5]
total_demand = sum(customer_demands.values()) * 365 * 4 
Trans_Cost = total_demand * total_miles * 0.06
print("Transportation cost is", Trans_Cost) 

WH_FC = warehouse_fixed_costs[2] + warehouse_fixed_costs[3] + warehouse_fixed_costs[4] + warehouse_fixed_costs[5]
print("Warehousing fixed cost is", WH_FC)

Total_Cost = Trans_Cost + WH_FC
print("Total Cost is", Total_Cost) 

Transportation cost is 2019705.5999999999
Warehousing fixed cost is 1468305
Total Cost is 3488010.5999999996
